In [2]:
%pip install opencv-python


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 215.0 kB/s eta 0:03:02
    --------------------------------------- 0.5/39.5 MB 215.0 kB/s eta 0:03:02
  


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import os

# Use raw string for Windows path
video_dir = r"D:\PHD KCL\Data"
output_dir = r"D:\PHD KCL\Frames"

# Create the output folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(video_dir):
    if filename.endswith(".mp4"):
        video_path = os.path.join(video_dir, filename)
        cap = cv2.VideoCapture(video_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_count = 0
        success, frame = cap.read()
        while success:
            if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % fps == 0:
                frame_filename = f"{filename[:-4]}_frame{frame_count}.jpg"
                frame_path = os.path.join(output_dir, frame_filename)
                cv2.imwrite(frame_path, frame)
                frame_count += 1
            success, frame = cap.read()
        cap.release()

print("✅ Frame extraction complete. All frames saved to D:\\PHD KCL\\Frames")


✅ Frame extraction complete. All frames saved to D:\PHD KCL\Frames


In [4]:
%pip install google-cloud-vision pandas


  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/526.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/526.1 kB ? eta -:--:--
   ------------------- -------------------- 262.1/526.1 kB ? eta -:--:--
   ---------------------------------------- 526.1/526.1 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.3 MB 2.0 MB/s eta 0:00:02
   --------- ------------------------------ 1.0/4.3 MB 1.9 MB/s eta 0:00:02
   -------------- ------------------------- 1.6/4.3 MB 2.0 MB/s eta 0:00:02
   ----------------- ---------------------- 1.8/4.3 MB 1.9 MB/s eta 0:00:02
   ------------------- -------------------- 2.1/4.3 MB 1.9 MB/s eta 0:00:02
   ------------------------ --------------- 2.6/4.3 MB 1.9 MB/s eta 0:00:01
   ----------------------------- ---------- 3.1/4.3 M


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import io
import pandas as pd
from google.cloud import vision

# Set path to your key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"D:\PHD KCL\google_vision_key.json"

# Initialize the API client
client = vision.ImageAnnotatorClient()

# Folder containing your frames
frames_dir = r"D:\PHD KCL\Frames"
output_data = []

# Loop through each frame
for filename in os.listdir(frames_dir):
    if filename.endswith(".jpg"):
        file_path = os.path.join(frames_dir, filename)
        with io.open(file_path, 'rb') as image_file:
            content = image_file.read()

        image = vision.Image(content=content)
        response = client.label_detection(image=image)
        labels = response.label_annotations

        for label in labels:
            output_data.append({
                'Image': filename,
                'Label': label.description,
                'Score': round(label.score, 3)
            })

print("✅ All frames analyzed successfully.")


✅ All frames analyzed successfully.


In [7]:
# Convert list to DataFrame
df = pd.DataFrame(output_data)

# Define CSV path and save it
csv_path = r"D:\PHD KCL\Output\vision_labels.csv"
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
df.to_csv(csv_path, index=False)

print(f"✅ CSV saved successfully at: {csv_path}")


✅ CSV saved successfully at: D:\PHD KCL\Output\vision_labels.csv
